## Projects:

- Diffusion Models;
- YOLO versions  - object detection;
- Segment Anything Model (https://segment-anything.com/) (Liked);
- Mathematical Integration;
- BERT fine-tuning - Text classification (compare with logistic regression and TF-IDF vectors);
- CNN visualization - review the different techniques used to analyze CNN predictions;
- Monte Carlo dropout - review briefly the concept of uncertainty and what are the main techniques used to deal with uncertainty in deep learning;
- Deep vs Wide Neural Networks - Try to understand the effect of adding layers or adding parameters in a layer;
- Overparameterization regime;
- Neural ODEs - Neural Networks and ordinary differential equations are the two main modelization techniques used nowadays, review papers explaining some evolution of the idea of Neural ODE.

General observations:

- Presentation of 15 min 
- 70% grade
- No need to submit the code

Evaluation aspects:

- Clarity
- Understanding of concepts
- Depth of research and/or experiments

In [2]:
import cv2
import numpy as np
import os
import torch
import matplotlib.pyplot as plt
from segment_anything import sam_model_registry, SamPredictor

# 📌 Paths to the ultrasound image and corresponding mask
dataset_path = "Breast_Images/"
image_path = os.path.join(dataset_path, "benign/benign (1).png")
mask_path = os.path.join(dataset_path, "benign/benign (1)_mask.png")

# Load the original ultrasound image
image = cv2.imread(image_path)
image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)  # Convert to RGB
image = cv2.resize(image, (1024, 1024))  # Resize to prevent crashes

# Load the mask image
mask = cv2.imread(mask_path, cv2.IMREAD_GRAYSCALE)  # Load as grayscale
mask = cv2.resize(mask, (1024, 1024))  # Ensure it matches the image size

# Find contours in the mask
contours, _ = cv2.findContours(mask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

# Ensure at least one contour is found
if len(contours) == 0:
    raise ValueError("No contours found in the mask.")

# Select the largest contour (assuming it's the tumor region)
largest_contour = max(contours, key=cv2.contourArea)

# Get bounding box around the largest contour
x, y, w, h = cv2.boundingRect(largest_contour)
bbox = np.array([x, y, x + w, y + h])  # Convert to (xmin, ymin, xmax, ymax)

# 📌 Load the SAM model
model_type = "vit_b"
sam_checkpoint = "sam_vit_b_01ec64.pth"
device = "cuda" if torch.cuda.is_available() else "cpu"

# Load SAM model
sam = sam_model_registry[model_type](checkpoint=sam_checkpoint).to(device)
predictor = SamPredictor(sam)

# Set the image in SAM
predictor.set_image(image)

# Expand dimensions for SAM input
bbox = np.expand_dims(bbox, axis=0)

# Run SAM prediction with detected bounding box
masks, _, _ = predictor.predict(
    box=bbox,
    point_coords=None,
    point_labels=None,
    multimask_output=False
)

# 📌 Display results
plt.figure(figsize=(15, 5))

# Original image
plt.subplot(1, 3, 1)
plt.imshow(image)
plt.title("Ultrasound Image")
plt.axis("off")

# Ground Truth Mask
plt.subplot(1, 3, 2)
plt.imshow(mask, cmap="gray")
plt.title("Ground Truth Mask")
plt.axis("off")

# Predicted Mask
plt.subplot(1, 3, 3)
plt.imshow(masks[0], cmap="gray")
plt.title("SAM Predicted Mask")
plt.axis("off")

plt.show()


FileNotFoundError: [Errno 2] No such file or directory: 'sam_vit_b_01ec64.pth'